### Train and Deploy a Neural Collaborative Filtering Model
In this notebook, you will execute code blocks to
1. inspect the training script ncf.py
2. train a model using Tensorflow Estimator
3. deploy and host the trained model as an endpoint using Amazon SageMaker Hosting Services
4. perform batch inference by calling the model endpoint

In [1]:

# In the last notebook (data-preparation-notebook.ipynb), we stored two variables.
# Let's restore those variables here. These variables are inputs for the model training process.

%store -r n_user
%store -r n_item

print(n_user)
print(n_item)

610
9724


In [12]:
# import requirements
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

In [ ]:
bucket_name = sagemaker_session.default_bucket()
print("default bucket name:", bucket_name)

In [ ]:
# specify the location of the training data
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')

In [4]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf.py'

"""

 Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 SPDX-License-Identifier: MIT-0

 Permission is hereby granted, free of charge, to any person obtaining a copy of this
 software and associated documentation files (the "Software"), to deal in the Software
 without restriction, including without limitation the rights to use, copy, modify,
 merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

"""

import tensorflow a

In [6]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.c5.2xlarge'     # type of instance to use for training

training_script = 'ncf.py'

training_parameters = {
    'epochs': 1,
    'batch_size': 256,
    'n_user': n_user,
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [13]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    train_instance_count=num_of_instance,
    train_instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distributions=distributed_training_spec,
    hyperparameters=training_parameters
)

In [ ]:
# kick off the training job
ncf_estimator.fit(training_data_uri)

#### Deploy the Endpoint

In [ ]:
# once the model is trained, we can deploy the model using Amazon SageMaker Hosting Services
# Here we deploy the model using one ml.c5.xlarge instance as a tensorflow-serving endpoint
# This enables us to invoke the endpoint like how we use Tensorflow serving
# Read more about Tensorflow serving using the link below
# https://www.tensorflow.org/tfx/tutorials/serving/rest_simple

endpoint_name = 'neural-collaborative-filtering-model-demo'

predictor = ncf_estimator.deploy(initial_instance_count=1,
                                 instance_type='ml.c5.xlarge',
                                 endpoint_type='tensorflow-serving',
                                 endpoint_name=endpoint_name)

#### Invoke

In [ ]:
# To use the endpoint in another notebook, we can initiate a predictor object as follows
from sagemaker.tensorflow import TensorFlowPredictor

predictor = TensorFlowPredictor(endpoint_name)

In [ ]:
# Define a function to read testing data
def _load_testing_data(base_dir):
    """ load testing data """
    df_test = np.load(os.path.join(base_dir, 'test.npy'))
    user_test, item_test, y_test = np.split(np.transpose(df_test).flatten(), 3)
    return user_test, item_test, y_test

In [ ]:
# read testing data from local
user_test, item_test, test_labels = _load_testing_data('./ml-latest-small/s3/')

# one-hot encode the testing data for model input
with tf.Session() as tf_sess:
    test_user_data = tf_sess.run(tf.one_hot(user_test, depth=n_user)).tolist()
    test_item_data = tf_sess.run(tf.one_hot(item_test, depth=n_item)).tolist()

# if you're using Tensorflow 2.0 for one hot encoding
# you can convert the tensor to list using:
# tf.one_hot(uuser_test, depth=n_user).numpy().tolist()

In [ ]:
# make batch prediction
batch_size = 100
y_pred = []
for idx in range(0, len(test_user_data), batch_size):
    # reformat test samples into tensorflow serving acceptable format
    input_vals = {
     "instances": [
         {'input_1': u, 'input_2': i}
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])
    ]}

    # invoke model endpoint to make inference
    pred = predictor.predict(input_vals)

    # store predictions
    y_pred.extend([i[0] for i in pred['predictions']])

In [ ]:
# let's see some prediction examples, assuming the threshold
# --- prediction probability view ---
print('This is what the prediction output looks like')
print(y_pred[:5], end='\n\n\n')

# --- user item pair prediction view, with threshold of 0.5 applied ---
pred_df = pd.DataFrame([
    user_test,
    item_test,
    (np.array(y_pred) >= 0.5).astype(int)],
).T

pred_df.columns = ['userId', 'movieId', 'prediction']

print('We can convert the output to user-item pair as shown below')
print(pred_df.head(), end='\n\n\n')

# --- aggregated prediction view, by user ---
print('Lastly, we can roll up the prediction list by user and view it that way')
print(pred_df.query('prediction == 1').groupby('userId').movieId.apply(list).head().to_frame(), end='\n\n\n')

#### Delete Endpoint

In [ ]:
# delete endpoint at the end of the demo
predictor.delete_endpoint(delete_endpoint_config=True)